# 02-rag.ipynb

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# %pip install pypdf langchain-community

## 저장 파트

In [ ]:
# 1. Document Load (PDF)
# 지원하는 문서 로더: https://docs.langchain.com/oss/python/integrations/document_loaders

from langchain_community.document_loaders import PyPDFLoader

# 불러올 파일 위치
file_path = './nke-10k-2023.pdf'

# 대상 pdf 를 변환해줄 로더
loader = PyPDFLoader(file_path)

# 로더가 pdf를 python에서 쓸 수 있도록 변환(pdf 1page -> 1 Document)
docs = loader.load()

print(len(docs))  # 원본 pdf 페이지수가 나옴

107


In [ ]:
# 2. Splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Document 를 잘라줄 스플리터
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

# 쪼개기
chunks = text_splitter.split_documents(docs)

print(len(chunks))  # 전체 chunk 개수
print(chunks[0].page_content)  # 첫번째 청크의 원본 텍스트 내용

516
Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31, 2023
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE TRANSITION PERIOD FROM                         TO                         .
Commission File No. 1-10635
NIKE, Inc.
(Exact name of Registrant as specified in its charter)
Oregon 93-0584541
(State or other jurisdiction of incorporation) (IRS Employer Identification No.)
One Bowerman Drive, Beaverton, Oregon 97005-6453
(Address of principal executive offices and zip code)
(503) 671-6453
(Registrant's telephone number, including area code)
SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:
Class B Common Stock NKE New York Stock Exchange
(Title of each class) (Trading symbol) (Name of each exchange on which registered)


In [ ]:
# 3. Embedding (숫자로 바꾸기)
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# 아래는 테스트용 (실제 텍스트 -> 벡터로 바뀌는 과정)
v1 = embeddings.embed_query(chunks[0].page_content)  # 청크1 벡터로 변환
v2 = embeddings.embed_query(chunks[1].page_content)  # 청크2 벡터로 변환

# 차원수는 같아야 한다.
print(len(v1) == len(v2))
print(v1[:10])  # 벡터 눈으로 확인하기

True
[0.053662512451410294, 0.04881863668560982, 0.018876856192946434, -0.004870585165917873, 0.025026675313711166, -0.008191843517124653, -0.006393199786543846, 0.024160001426935196, -0.0011627372587099671, 0.010803735814988613]


In [ ]:
# 4. Vector Store 에 저장하기
from langchain_core.vectorstores import InMemoryVectorStore

# 테스트/개발용 메모리 벡터스토어
vector_store = InMemoryVectorStore(embeddings)

# pdf 쪼개놓은 chunks 를 벡터스토어에 저장 (저장 후 id들이 나옴)
ids = vector_store.add_documents(documents=chunks)

## 검색 파트

In [ ]:
# 5.